## FM ver5 구현
### - 변수 추가 : user_id + item_id + location + author + age + year
### - optimizer : adam optimizer
### - 최종 예측 값 보정 : 1 보다 작은 값, 10 보다 큰 값 보정

### 1. 데이터셋 로드 

In [179]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

u_cols = ['user_id', 'location', 'age']
users = pd.read_csv('..\\0. dataset\\BX-Users.csv', encoding='latin-1')
users.columns = u_cols

i_cols = ['item_id', 'title' ,'author','year', 'publisher', 'img_s', 'img_m', 'img_l']
items = pd.read_csv('..\\0. dataset\\BX-Books.csv', encoding='latin-1')
items.columns = i_cols
items = items[['item_id', 'title' ,'author','year', 'publisher']]

r_cols = ['user_id', 'item_id', 'rating']
ratings = pd.read_csv('..\\0. dataset\\BX-Book-Ratings.csv', encoding='latin-1')
ratings.columns = r_cols

### 2. 기본적인 전처리 수행
#### - 평점 0 제거, 연속형 변수 re-scaling

In [180]:
# 사용자가 평가한 아이템 평점 데이터에서 평점이 0인 데이터 제거
ratings = ratings.drop(ratings[ratings['rating'] == 0].index)

# 'age' 열의 값을 50으로 나누어 re-scaling
users['age'] = users['age'] / 50

# 'year' 열의 값을 1990으로 나누어 re-scaling
items['year'] = items['year'] / 1990

# DataFrame을 병합
ratings = pd.merge(ratings, items, how='inner', on='item_id')
ratings = pd.merge(ratings, users, how='inner', on='user_id')

# 'ratings' DataFrame의 행을 무작위로 섞음
ratings = shuffle(ratings)

### 3. 변수 및 데이터셋 인코딩

In [181]:
# User encoding
user_dict = {}
for i in set(ratings['user_id']):
    user_dict[i] = len(user_dict)
n_user = len(user_dict)

# Item encoding
item_dict = {}
start_point = n_user
for i in set(ratings['item_id']):
    item_dict[i] = start_point + len(item_dict)
n_item = len(item_dict)
start_point += n_item

# Location encoding
location_dict = {}
for i in set(ratings['location']):
    location_dict[i] = start_point + len(location_dict)
n_location = len(location_dict)
start_point += n_location

# Author encoding
author_dict = {}
for i in set(ratings['author']):
    author_dict[i] = start_point + len(author_dict)
n_author = len(author_dict)
start_point += n_author

age_index = start_point
start_point += 1

year_index = start_point
start_point += 1

num_x = start_point               # Total number of x

# Generate X data                                   
# sparse matrix 로 변환하는 작업
x = []
y = []
w0 = np.mean(ratings['rating'])
for i in range(len(ratings)):
    case = ratings.iloc[i]                        
    x_index = []
    x_value = []
    x_index.append(user_dict[case['user_id']])    # User id encoding. 해당 user id의 일력번호를 가져옴
    x_value.append(1)
    x_index.append(item_dict[case['item_id']])    # item id encoding. 해당 item id의 일력번호를 가져옴
    x_value.append(1)
    x_index.append(location_dict[case['location']])
    x_value.append(1)
    x_index.append(author_dict[case['author']])
    x_value.append(1)
    x_index.append(age_index)
    x_value.append(case['age'])
    x_index.append(year_index)
    x_value.append(case['year'])
    x.append([x_index, x_value])
    y.append(case['rating'] - w0)
    if (i % 10000) == 0:
        print('Encoding ', i, ' cases...')

Encoding  0  cases...
Encoding  10000  cases...
Encoding  20000  cases...
Encoding  30000  cases...


### 3. FM 클래스 정의

In [182]:
# RMSE (Root Mean Square Error) 계산 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))

# Factorization Machine Class
class FM():
    #  초기화 함수
    def __init__(self, N, K, train_x, test_x, train_y, test_y, alpha=0.001, beta1=0.9, beta2=0.999, beta=0.01, epsilon=1e-8,
                 iterations=100, tolerance=0.005, l2_reg=True, verbose=True):
        self.K = K                      # Number of latent factors
        self.N = N                      # Number of x (variables)
        self.alpha = alpha              # 학습률
        self.beta = beta                # L2 정규화 항
        self.iterations = iterations    # 반복 횟수
        self.l2_reg = l2_reg            # L2 정규화 사용 여부
        self.tolerance = tolerance      # 조기 종료를 위한 허용 오차 범위
        self.verbose = verbose          # 진행 상황 출력 여부

        self.beta1 = beta1              # 모멘텀 감쇠율
        self.beta2 = beta2              # 스케일 조정된 경사 제곱의 감쇠율
        self.epsilon = epsilon          # 수치 안정성을 위한 작은 상수

        # Adam 매개변수 초기화
        self.m_w, self.v_w = None, None  # w에 대한 모멘텀과 스케일 조정된 경사 제곱
        self.m_v, self.v_v = None, None  # v에 대한 모멘텀과 스케일 조정된 경사 제곱

        # w 초기화
        self.w = np.random.normal(scale=1. / self.N, size=(self.N))
        # v 초기화
        self.v = np.random.normal(scale=1. / self.K, size=(self.N, self.K))

        self.train_x, self.test_x, self.train_y, self.test_y = train_x, test_x, train_y, test_y

    def test(self):  # Training 하면서 RMSE 계산
        # ADAM을 iterations 숫자만큼 수행
        best_RMSE = 10000
        training_process = []
        for i in range(self.iterations):
            rmse1 = self.adam(self.train_x, self.train_y)  # ADAM & Train RMSE 계산
            rmse2 = self.test_rmse(self.test_x, self.test_y)  # Test RMSE 계산
            training_process.append((i, rmse1, rmse2))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("\tIteration: %d ; Train RMSE = %.6f ; Test RMSE = %.6f" % (i + 1, rmse1, rmse2))
            if best_RMSE > rmse2:  # New best record
                best_RMSE = rmse2
            elif (rmse2 - best_RMSE) > self.tolerance:  # RMSE is increasing over tolerance
                break

        return best_RMSE

    # w, v 업데이트를 위한 adam optimizer
    def adam(self, x_data, y_data):
        y_pred = []
        for data, y in zip(x_data, y_data):
            x_idx = data[0]
            x_0 = np.array(data[1])
            x_1 = x_0.reshape(-1, 1)

            # 기존 코드와 동일한 방식으로 점수 계산
            bias_score = np.sum(self.w[x_idx] * x_0)
            vx = self.v[x_idx] * x_1
            sum_vx = np.sum(vx, axis=0)
            sum_vx_2 = np.sum(vx * vx, axis=0)
            latent_score = 0.5 * np.sum(np.square(sum_vx) - sum_vx_2)
            y_hat = bias_score + latent_score
            y_pred.append(y_hat)

            error = y - y_hat
            # Adam 업데이트 부분
            # 1. 경사 계산
            grad_w = error * x_0 - self.beta * self.w[x_idx] if self.l2_reg else error * x_0
            grad_v = error * ((x_1) * sum_vx - (vx * x_1)) - self.beta * self.v[x_idx] if self.l2_reg else error * (
                        (x_1) * sum_vx - (vx * x_1))

            # 2. m, v의 초기화
            if self.m_w is None:
                self.m_w, self.v_w = np.zeros_like(self.w), np.zeros_like(self.w)
                self.m_v, self.v_v = np.zeros_like(self.v), np.zeros_like(self.v)

            # 3. 모멘텀과 스케일 조정된 경사 제곱 업데이트
            self.m_w[x_idx] = self.beta1 * self.m_w[x_idx] + (1 - self.beta1) * grad_w
            self.m_v[x_idx] = self.beta1 * self.m_v[x_idx] + (1 - self.beta1) * grad_v
            self.v_w[x_idx] = self.beta2 * self.v_w[x_idx] + (1 - self.beta2) * np.square(grad_w)
            self.v_v[x_idx] = self.beta2 * self.v_v[x_idx] + (1 - self.beta2) * np.square(grad_v)

            # 4. 편향 보정된 매개변수 업데이트
            m_w_hat = self.m_w[x_idx] / (1 - self.beta1)
            m_v_hat = self.m_v[x_idx] / (1 - self.beta1)
            v_w_hat = self.v_w[x_idx] / (1 - self.beta2)
            v_v_hat = self.v_v[x_idx] / (1 - self.beta2)
            self.w[x_idx] += self.alpha * m_w_hat / (np.sqrt(v_w_hat) + self.epsilon)
            self.v[x_idx] += self.alpha * m_v_hat / (np.sqrt(v_v_hat) + self.epsilon)
        return RMSE(y_data, y_pred)

    def test_rmse(self, x_data, y_data):
        y_pred = []
        for data, y in zip(x_data, y_data):
            y_hat = self.predict(data[0], data[1])
            
            # y_hat에 전역 바이어스 w0 추가
            y_hat += w0
            
            # 예측값이 특정 범위(1에서 10)를 벗어나지 않도록 조정
            if y_hat > 10:
                y_hat = 10
            elif y_hat < 1:
                y_hat = 1
            
            # 조정된 예측값을 리스트에 추가    
            y_pred.append(y_hat)
        
        # 실제 데이터에 w0를 더한 값과 예측값 사이의 RMSE 계산 및 반환
        return RMSE(y_data + w0, y_pred)

    def predict(self, idx, x):
        x_0 = np.array(x)
        x_1 = x_0.reshape(-1, 1)

        # biases
        bias_score = np.sum(self.w[idx] * x_0)

        # score 계산
        vx = self.v[idx] * (x_1)
        sum_vx = np.sum(vx, axis=0)
        sum_vx_2 = np.sum(vx * vx, axis=0)
        latent_score = 0.5 * np.sum(np.square(sum_vx) - sum_vx_2)

        # 예측값 계산
        y_hat = bias_score + latent_score
        return y_hat

### 4. 모형 학습 및 평가
#### - test/train set을 교체하며 3회 측정하여 평균  

In [183]:
K = 350             # 모델 하이퍼파라미터 설정
results = []        # 결과를 저장할 리스트

n = len(x)          # 데이터의 전체 길이
fold_size = n // 3  # 각 폴드의 크기

# 3회의 실험 진행 (데이터를 3등분하여 2:1의 비율로 train/test set 을 번갈아가며 적용)
for i in range(3):
    print(f'< trial #{i+1} >')

    # 테스트 세트의 시작과 끝 인덱스 계산
    start_index = i * fold_size
    end_index = start_index + fold_size if i < 2 else n

    # 훈련 및 테스트 데이터 생성
    test_x = x[start_index:end_index]
    test_y = y[start_index:end_index]
    train_x = x[:start_index] + x[end_index:]
    train_y = y[:start_index] + y[end_index:]

    # FM 모델 초기화 및 파라미터 설정
    fm1 = FM(num_x, K, train_x, test_x, train_y, test_y, alpha=0.00005, beta=0.0075, beta1=0.75, beta2=0.95, iterations=900, tolerance=0.0001, l2_reg=True, verbose=True)
    
    # 모델 테스트 및 결과 저장
    result = fm1.test()
    results.append(result)
    
    print()
    print(f'\tbest_RMSE = {result}')
    print() 

# 각 실험의 RMSE 결과 출력
print()
print('********************* final results *********************')
print()
for i in range(len(results)):
    print(f'RMSE {i+1} : {results[i]}')

# 평균 RMSE 계산 및 출력
print(f'average RMSE : {np.mean(results)}')

< trial #1 >
	Iteration: 10 ; Train RMSE = 1.508000 ; Test RMSE = 1.654558
	Iteration: 20 ; Train RMSE = 1.351205 ; Test RMSE = 1.627063
	Iteration: 30 ; Train RMSE = 1.254879 ; Test RMSE = 1.621441

	best_RMSE = 1.621314389440082

< trial #2 >
	Iteration: 10 ; Train RMSE = 1.490769 ; Test RMSE = 1.668639
	Iteration: 20 ; Train RMSE = 1.330293 ; Test RMSE = 1.647355

	best_RMSE = 1.64528545899976

< trial #3 >
	Iteration: 10 ; Train RMSE = 1.501257 ; Test RMSE = 1.652465
	Iteration: 20 ; Train RMSE = 1.344153 ; Test RMSE = 1.629122
	Iteration: 30 ; Train RMSE = 1.248543 ; Test RMSE = 1.625073

	best_RMSE = 1.6250731716717834


********************* final results *********************

RMSE 1 : 1.621314389440082
RMSE 2 : 1.64528545899976
RMSE 3 : 1.6250731716717834
average RMSE : 1.6305576733705418
